# Document scanning tool

### Imports

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import argparse
from dl_docs import *

%load_ext autoreload
%autoreload 2

## Download docs from 3gpp

In [ ]:
url = 'https://www.3gpp.org/ftp/Specs/2023-06'  # URL of the webpage to be parsed

# Fetch links to zip files
# zip_urls = extract_zip_urls(url)
# print(zip_urls)

# Download zip files
download_docs_from_3gpp(url)